# An Introduction to Linear Learner with MNIST
_**Making a Binary Prediction of Whether a Handwritten Digit is a 0**_

1. [Introduction](#Introduction)
2. [Prerequisites and Preprocessing](#Prequisites-and-Preprocessing)
  1. [Permissions and environment variables](#Permissions-and-environment-variables)
  2. [Data ingestion](#Data-ingestion)
  3. [Data inspection](#Data-inspection)
  4. [Data conversion](#Data-conversion)
3. [Training the linear model](#Training-the-linear-model)
4. [Set up hosting for the model](#Set-up-hosting-for-the-model)
5. [Validate the model for use](#Validate-the-model-for-use)


## Introduction
Welcome to our example introducing Amazon SageMaker's Linear Learner Algorithm!  Today, we're analyzing the [MNIST](https://en.wikipedia.org/wiki/MNIST_database) dataset which consists of images of handwritten digits, from zero to nine.  We'll use the individual pixel values from each 28 x 28 grayscale image to predict a yes or no label of whether the digit is a 0 or some other digit (1, 2, 3, ... 9).

The method that we'll use is a linear binary classifier.  Linear models are supervised learning algorithms used for solving either classification or regression problems.  As input, the model is given labeled examples ( **`x`**, `y`). **`x`** is a high dimensional vector and `y` is a numeric label.  Since we are doing binary classification, the algorithm expects the label to be either 0 or 1 (but Amazon SageMaker Linear Learner also supports regression on continuous values of `y`).  The algorithm learns a linear function, or linear threshold function for classification, mapping the vector **`x`** to an approximation of the label `y`.

Amazon SageMaker's Linear Learner algorithm extends upon typical linear models by training many models in parallel, in a computationally efficient manner.  Each model has a different set of hyperparameters, and then the algorithm finds the set that optimizes a specific criteria.  This can provide substantially more accurate models than typical linear algorithms at the same, or lower, cost.

To get started, we need to set up the environment with a few prerequisite steps, for permissions, configurations, and so on.


## 소개
Amazon SageMaker의 Linear Learner을 소개하는 예제에 오신 것을 환영합니다. 오늘, 우리는 0부터 9까지의 필기체 숫자로 구성된 [MNIST](https://en.wikipedia.org/wiki/MNIST_database) 데이터셋을 분석해보겠습니다. 각 28 x 28 그레이 스케일 이미지의 개별 픽셀 값을 사용하여 이미지가 0부터 9까지 숫자에 대한 yes 또는 no 레이블을 예측합니다. 

우리가 사용할 방법은 선형 이진 분류기입니다. 선형 모델은 분류 또는 회귀 문제를 해결하는 데 사용되는 지도학습 알고리즘입니다. 입력으로는 모델에서 레이블이 지정된 예제 ( **`x`**, `y`)가 제공됩니다. **`x`** 는 고차원 벡터이며 `y` 는 숫자 레이블입니다. 이진 분류를 수행하기 때문에 알고리즘은 레이블이 0 또는 1 일 것으로 예상합니다(그러나 Amazon Sagemaker Linear Learner는 `y`의 연속 값에 대한 회귀도 지원합니다). 이 알고리즘은 벡터 **`x`** 를 레이블 `y`의 근사값에 매핑하여 선형 함수 또는 분류를 위한 선형 임계 값 함수를 학습합니다. 

Amazon SageMaker의 Linear Learner 알고리즘은 계산 효율적인 방식으로 많은 모델을 병렬로 학습하여 일반적인 선형 모델을 확장합니다. 각 모델에는 서로 다른 하이퍼파라메터 세트가 있으며 알고리즘은 특정 기준을 최적화하는 세트를 찾습니다. 이는 동일하거나 더 낮은 비용으로 일반적인 선형 알고리즘보다 훨씬 더 정확한 모델을 제공할 수 있습니다. 

시작하려면 권한, 구성 등을 위한 몇 가지 젠제 조건 단계로 환경을 설정해야 합니다. 


## Prequisites and Preprocessing

### Permissions and environment variables

_This notebook was created and tested on an ml.m4.xlarge notebook instance._

Let's start by specifying:

- The S3 bucket and prefix that you want to use for training and model data.  This should be within the same region as the Notebook Instance, training, and hosting.
- The IAM role arn used to give training and hosting access to your data. See the documentation for how to create these.  Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the boto regexp with a the appropriate full IAM role arn string(s).

## 전제 조건 및 전처리

### 권한 및 환경 변수

_이 노트북은 ml.m4.xlarge 노트북 인스턴스에서 작성 및 테스트되었습니다._

다음을 지정하여 시작하겠습니다.

- 훈련 및 모델 데이터에 사용하려는 S3 버킷 및 접두사, 노트북 인스턴스, 훈련 및 호스팅은 동일 리전에 있어야 합니다. 
- 훈련 및 호스팅, 데이터 엑세스에 사용하는 IAM 역할 arn을 생성하는 방법에 대한 설명서를 참조하세요. 노트북 인스턴스, 훈련 및 호스팅에 두개 이상의 역할이 필요한 경우 boto regexp를 적절한 IAM 역할 arn 문자열로 바꿔주세요.

In [ ]:
bucket = '<your_s3_bucket_name_here>'
prefix = 'sagemaker/DEMO-linear-mnist'
 
# Define IAM role
import boto3
import re
from sagemaker import get_execution_role

role = get_execution_role()

### Data ingestion

Next, we read the dataset from an online URL into memory, for preprocessing prior to training. This processing could be done *in situ* by Amazon Athena, Apache Spark in Amazon EMR, Amazon Redshift, etc., assuming the dataset is present in the appropriate location. Then, the next step would be to transfer the data to S3 for use in training. For small datasets, such as this one, reading into memory isn't onerous, though it would be for larger datasets.

### 데이터 수집

다음, 훈련 전에 전처리를 위해 온라인 URL에서 메모리로 데이터 세트를 읽습니다. 이 절차는 데이터 세트가 적절한 위치에 있다고 가정 할 때 Amazon Athena, Amazon EMR의 Apache Spark, Amazon Redshift 등*에서 수행* 할 수 있습니다. 다음 단계는 훈련에 사용하기 위해 데이터를 S3로 전송하는 것입니다. 큰 데이터 세트에 대해서는 메모리로 읽는 것이 번거롭겠지만 이와 같은 작은 데이터 세트에서는 그렇지 않습니다.


In [ ]:
%%time
import pickle, gzip, numpy, urllib.request, json

# Load the dataset
urllib.request.urlretrieve("http://deeplearning.net/data/mnist/mnist.pkl.gz", "mnist.pkl.gz")
with gzip.open('mnist.pkl.gz', 'rb') as f:
    train_set, valid_set, test_set = pickle.load(f, encoding='latin1')

### Data inspection

Once the dataset is imported, it's typical as part of the machine learning process to inspect the data, understand the distributions, and determine what type(s) of preprocessing might be needed. You can perform those tasks right here in the notebook. As an example, let's go ahead and look at one of the digits that is part of the dataset.

### 데이터 검사

데이터 세트를 가져온 후에는 데이터를 검사하고 분포를 이해하며 필요한 전처리 유형을 결정하는 일반적인 머신 러닝 프로세스입니다. 이런 작업들은 바로 노트북에서 수행할 수 있습니다. 예를 들어, 데이터 세트의 일부인 숫자 중 하나를 살펴 보겠습니다.


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (2,10)


def show_digit(img, caption='', subplot=None):
    if subplot==None:
        _,(subplot)=plt.subplots(1,1)
    imgr=img.reshape((28,28))
    subplot.axis('off')
    subplot.imshow(imgr, cmap='gray')
    plt.title(caption)

show_digit(train_set[0][30], 'This is a {}'.format(train_set[1][30]))

### Data conversion

Since algorithms have particular input and output requirements, converting the dataset is also part of the process that a data scientist goes through prior to initiating training. In this particular case, the Amazon SageMaker implementation of Linear Learner takes recordIO-wrapped protobuf, where the data we have today is a pickle-ized numpy array on disk.

Most of the conversion effort is handled by the Amazon SageMaker Python SDK, imported as `sagemaker` below.

### 데이터 변환

알고리즘에는 특정 입력 및 출력 요구 사항이 있으므로 데이터 세트 변환은 데이터 과학자가 훈련을 시작하기 전에 거치는 프로세스의 일부이기도합니다. 이 특별한 경우, Linear Learner의 Amazon SageMaker 구현은 recordIO-wrapped protobuf를 사용합니다. 오늘 사용할 데이터는 디스크에 있는 pickle-ized numpy 배열입니다.

대부분의 변환은 아래에서 import한 `sagemaker`, Amazon SageMaker Python SDK에 의해 처리됩니다.

In [ ]:
import io
import numpy as np
import sagemaker.amazon.common as smac

vectors = np.array([t.tolist() for t in train_set[0]]).astype('float32')
labels = np.where(np.array([t.tolist() for t in train_set[1]]) == 0, 1, 0).astype('float32')

buf = io.BytesIO()
smac.write_numpy_to_dense_tensor(buf, vectors, labels)
buf.seek(0)

## Upload training data
Now that we've created our recordIO-wrapped protobuf, we'll need to upload it to S3, so that Amazon SageMaker training can use it.

## 훈련 데이터 업로드
RecordIO-wrapped protobuf를 생성하였고 이제 Amazon SageMaker 훈련에서 사용할 수 있도록 S3에 업로드해야합니다.


In [ ]:
import boto3
import os

key = 'recordio-pb-data'
boto3.resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train', key)).upload_fileobj(buf)
s3_train_data = 's3://{}/{}/train/{}'.format(bucket, prefix, key)
print('uploaded training data location: {}'.format(s3_train_data))

Let's also setup an output S3 location for the model artifact that will be output as the result of training with the algorithm.

알고리즘 학습의 결과로 출력 될 모델 아티팩트에 대한 출력 S3 위치를 설정해봅시다.

In [ ]:
output_location = 's3://{}/{}/output'.format(bucket, prefix)
print('training artifacts will be uploaded to: {}'.format(output_location))

## Training the linear model

Once we have the data preprocessed and available in the correct format for training, the next step is to actually train the model using the data. Since this data is relatively small, it isn't meant to show off the performance of the Linear Learner training algorithm, although we have tested it on multi-terabyte datasets.

Again, we'll use the Amazon SageMaker Python SDK to kick off training, and monitor status until it is completed.  In this example that takes between 7 and 11 minutes.  Despite the dataset being small, provisioning hardware and loading the algorithm container take time upfront.

First, let's specify our containers.  Since we want this notebook to run in all 4 of Amazon SageMaker's regions, we'll create a small lookup.  More details on algorithm containers can be found in [AWS documentation](https://docs-aws.amazon.com/sagemaker/latest/dg/sagemaker-algo-docker-registry-paths.html).

## 선형 모델 훈련

훈련을 위한 올바른 형식으로 데이터를 사전 처리가 완료되면 다음 단계로 실제로 데이터를 사용하여 모델을 훈련합니다. 테라 바이트 급 데이터 세트에서도 테스트했으나 이 데이터는 상대적으로 작기 때문에 Linear Learner 훈련 알고리즘의 성능을 표현하려고하는 것은 아닙니다.

Amazon SageMaker Python SDK를 사용하여 훈련을 시작하고 완료 될 때까지 상태를 모니터링합니다. 이 예제는 7 ~ 11분이 걸립니다. 데이터 세트가 작으나 하드웨어 프로비저닝 및 알고리즘 컨테이너를 로드하는데 시간이 걸립니다.

먼저 컨테이너를 지정합니다. 이 노트북이 4개의 Amazon SageMaker를 지원하는 리전에서 모두 실행되기를 원하므로 컨테이너 URI를 불러옵니다. 알고리즘 컨테이너에 대한 자세한 내용은 [AWS documentation](https://docs-aws.amazon.com/sagemaker/latest/dg/sagemaker-algo-docker-registry-paths.html)에서 확인할 수 있습니다.

In [ ]:
from sagemaker.amazon.amazon_estimator import get_image_uri
container = get_image_uri(boto3.Session().region_name, 'linear-learner')

Next we'll kick off the base estimator, making sure to pass in the necessary hyperparameters.  Notice:
- `feature_dim` is set to 784, which is the number of pixels in each 28 x 28 image.
- `predictor_type` is set to 'binary_classifier' since we are trying to predict whether the image is or is not a 0.
- `mini_batch_size` is set to 200.  This value can be tuned for relatively minor improvements in fit and speed, but selecting a reasonable value relative to the dataset is appropriate in most cases.

다음 기본 estimator를 시작하여 필요한 하이퍼파라미터를 전달합니다.
비고:
- `feature_dim` 은 784로 설정되는데, 이는 28 x 28 이미지의 픽셀 수입니다.
- `predictor_type` 이미지가 0 인지 아닌지를 예측하려고하기 때문에 'binary_classifier'로 설정됩니다. 
- `mini_batch_size`는 200으로 설정됩니다. 이 값은 적합도 및 속도가 약간 향상되도록 조정할 수 있지만 대부분의 경우 데이터 세트에 의해 합리적인 값을 선택하는 것이 적합합니다.

In [ ]:
import boto3
import sagemaker

sess = sagemaker.Session()

linear = sagemaker.estimator.Estimator(container,
                                       role, 
                                       train_instance_count=1, 
                                       train_instance_type='ml.c4.xlarge',
                                       output_path=output_location,
                                       sagemaker_session=sess)
linear.set_hyperparameters(feature_dim=784,
                           predictor_type='binary_classifier',
                           mini_batch_size=200)

linear.fit({'train': s3_train_data})

## Set up hosting for the model
Now that we've trained our model, we can deploy it behind an Amazon SageMaker real-time hosted endpoint.  This will allow out to make predictions (or inference) from the model dyanamically.

_Note, Amazon SageMaker allows you the flexibility of importing models trained elsewhere, as well as the choice of not importing models if the target of model creation is AWS Lambda, AWS Greengrass, Amazon Redshift, Amazon Athena, or other deployment target._

## 모델 호스팅 설정
모델 훈련을 마쳤으므로 Amazon SageMaker 실시간 호스팅 엔트포인드 뒤에 모델을 배포할 수 있습니다. 이것은 모델에서 동적으로 예측 (또는 추론) 할수 있게합니다.

_Amazon SageMaker를 사용하면 다른 곳에서 학습한 모델을 유연하게 import 할 수 있으나 모델 생성 대상이 AWS Lambda, AWS Greengrass, Amazon Redshift, Amazon Athena 또는 기타 배포 대상인 경우 모델을 import 하지 않을 수 있습니다._

In [ ]:
linear_predictor = linear.deploy(initial_instance_count=1,
                                 instance_type='ml.m4.xlarge')

## Validate the model for use
Finally, we can now validate the model for use.  We can pass HTTP POST requests to the endpoint to get back predictions.  To make this easier, we'll again use the Amazon SageMaker Python SDK and specify how to serialize requests and deserialize responses that are specific to the algorithm.

## 사용하기위한 모델 검증
마지막으로 모델의 유효성을 검사 할 수 있습니다. 예측 결과를 얻기 위해 HTTP POST 요청을 엔드포인트로 전달할 수 있습니다. 이를보다 쉽게하기 위해 Amazon SageMaker Python SDK를 다시 사용하고 알고리즘과 관련된 요청을 serialize하고 응답을 deserialize하는 방법을 지정합니다. 

In [ ]:
from sagemaker.predictor import csv_serializer, json_deserializer

linear_predictor.content_type = 'text/csv'
linear_predictor.serializer = csv_serializer
linear_predictor.deserializer = json_deserializer

Now let's try getting a prediction for a single record.

이제 단일 레코드에 대한 예측을 진행해봅니다.

In [ ]:
result = linear_predictor.predict(train_set[0][30:31])
print(result)

단일 레코드에 대한 예측이 완료하였습니다. 하나의 레코드에 대해 엔드 포인트가`score` 및`predicted_label`을 포함하여`predictions`를 포함하는 일부 JSON을 리턴했음을 알 수 있습니다. 이 경우 `score`는 숫자가 0일 확률을 나타내는 [0, 1] 사이의 연속적인 값입니다. `predicted_label`은`0` 또는`1`의 값을 취합니다. 여기서 `1`은 (직관적으로) 이미지가 0임을 예측하고,`0`은 이미지가 0이 아니라고 예측하고 있음을 나타냅니다.

전체 이미지 배치를 수행하고 예측 정확도를 평가합시다.

OK, a single prediction works.  We see that for one record our endpoint returned some JSON which contains `predictions`, including the `score` and `predicted_label`.  In this case, `score` will be a continuous value between [0, 1] representing the probability we think the digit is a 0 or not.  `predicted_label` will take a value of either `0` or `1` where (somewhat counterintuitively) `1` denotes that we predict the image is a 0, while `0` denotes that we are predicting the image is not of a 0.

Let's do a whole batch of images and evaluate our predictive accuracy.

In [ ]:
import numpy as np

predictions = []
for array in np.array_split(test_set[0], 100):
    result = linear_predictor.predict(array)
    predictions += [r['predicted_label'] for r in result['predictions']]

predictions = np.array(predictions)

In [ ]:
import pandas as pd

pd.crosstab(np.where(test_set[1] == 0, 1, 0), predictions, rownames=['actuals'], colnames=['predictions'])

위의 혼동 행렬에서 알 수 있듯이 931개 0 이미지를 올바르게 예측한 반면 44개 0이 아닌 이미지를 0으로 예측했고 49개 0인 이미지를 예측하지 못했습니다.  

As we can see from the confusion matrix above, we predict 931 images of 0 correctly, while we predict 44 images as 0s that aren't, and miss predicting 49 images of 0.

### (선택 사항) 엔드포인트 삭제

이 노트북을 완료했다면 아래 셀에서 delete_endpoint 라인을 실행하세요. 생성된 호스팅 엔드포인트가 제거되고 남겨진 인스턴스에서 비용 발생하는 것을 방지할 수 있습니다.

### (Optional) Delete the Endpoint

If you're ready to be done with this notebook, please run the delete_endpoint line in the cell below.  This will remove the hosted endpoint you created and avoid any charges from a stray instance being left on.

In [ ]:
import sagemaker

sagemaker.Session().delete_endpoint(linear_predictor.endpoint)